In [1]:
import numpy as np
import matplotlib.pyplot as plt

# TODO 1: Source Model

In [2]:
# Some auxiliary functions
def make_batch(mean_B : float) -> int:
    '''
    Returns an integer drawn by a geometric distribution with parameter p=1/mean_B.
    '''
    p = 1/mean_B # "Success" probability
    B = np.random.geometric(p)
    return B

def wait_T_off(mean_T_off : float) -> float:
    '''
    Returns a float number drawn by an exponential distribution with parameter lambda=1/mean_T_off
    '''
    T_off = np.random.exponential(mean_T_off)
    return T_off

def gen_VBR_traffic(arr_times, batches, T_on : float, tau : float, mean_B : int, T_off : float) -> None:
    '''
    Function that generates VBR traffic with the following parameters:
    + arr_times: list to store the times of every new batch arrival.
    + batches:   list to store the batch sizes at every arrival time.
    + T_on:      duration of the ON period.
    + tau:       interarrival time during the ON period.
    + mean_B:    mean value for the size of a batch.
    + T_off:     mean duration of the OFF period
    
    Returns:
    + Updated arr_times list.
    + Updated batches list.
    '''
    # FIRST BATCH ARRIVAL
    # At this point arr_times will have one element more than batches, since the information
    # about the next arriving packet is given by knowing the duration of T_off.
    t_on = 0
    batches.append(make_batch(mean_B)) # Batch size of the batch arriving at t_on=0.
    
    t_on += tau # arrival time of the second batch of the ON period
    while t_on <= T_on:
        arr_times.append(tau)
        batches.append(make_batch(mean_B))
        t_on += tau
    
    # Compute the "wasted time" in the ON period
    if (T_on/tau).is_integer():
        arr_times.append(wait_T_off(T_off))
    else:
        wasted_time = T_on - (t_on - tau) # Can only be > 0
        arr_times.append(wasted_time + wait_T_off(T_off))
    
    return None

In [3]:
# Generate N_c ON/OFF cycles of traffic
def source_model(arr_times, batches, T_on, tau, mean_B, T_off, L, Nc):
    '''
    Extends functioning of the above function to the whole source model
    '''
    t0 = 0 # Initial Time of the first ON period (w.l.o.g. = 0)
    arr_times.append(t0)
    
    cycle_count = 1
    while cycle_count <= Nc:
        gen_VBR_traffic(arr_times, batches, T_on, tau, mean_B, T_off)
        cycle_count += 1
    
    # pop last element of the arr_times vector since the last cycle has to be cut off
    arr_times.pop()
    
    # Convert lists in np.array
    arr_times = np.array(arr_times)
    batches = np.array(batches)
    
    # Workload vector
    workload = np.multiply(batches, L)
    return arr_times, batches, workload

In [4]:
# Write matrix in fixed point notation
def write_matrix_S(interarr_times, batches, workload, filename='S_matrix.txt'):
    '''
    Function that take the 3 array generated by source_model and writes a file
    with the corresponding matrix S in fixed point notation
    '''
    # Reshape all the arrays
    interarr_times = interarr_times.reshape((-1,1))
    batches = batches.reshape((-1,1))
    workload = workload.reshape((-1,1))
    # Create matrix S (with 3 columns)
    S = np.column_stack((interarr_times, batches, workload))
    # Fixed point notation
    np.set_printoptions(suppress=True, formatter={'float_kind':'{:0.6f}'.format})
    S = np.around(S, 6)
    # Write on txt file
    np.savetxt(filename, S, fmt='%0.6f')
    return S

In [5]:
# Final function for the Source Module
def source_model_and_S(arr_times, batches, T_on, tau, mean_B, T_off, L, Nc, filename='S_matrix.txt'):
    '''
    Function that does all the tasks of section 1.1 of the HW Assignment.
    It computes interarrival times, batch sizes and workload, and store them in a matrix
    S, which is also written in a .txt file in fixed point notation (with 6 digits precision)
    '''
    arr_times, batches, workload = source_model(arr_times, batches, T_on, tau, mean_B, T_off, L, Nc)
    S = write_matrix_S(arr_times, batches, workload, filename)
    return S

In [6]:
arr_times, batches = [], []
S = source_model_and_S(arr_times, batches,T_on=1,tau=0.2,mean_B=3,T_off=1,L=1000,Nc=10000)

# TODO 2: Token Bucket Filter

In [17]:
# Auxiliary Functions
def read_matrix_S(fname = 'S_matrix.txt'):
    S = np.loadtxt(fname, delimiter=' ')
    return S

S = read_matrix_S()
S = S[:,1]
unique, counts = np.unique(S, return_counts=True)
print(unique), print(counts)

def count_compliant(F : list, S : np.ndarray,  t : int, bt : int):
    '''
    Function that writes in F a 1 for each compliant pck, and a 0 for each non compliant pck
    that occur at a certain arrival event (each row of S).
    bt is the number of tokens available at that time.
    It also returns the updated bt capacity.
    '''
    # Extract info corresponding to the t-th row of S
    batch_info = S[t,:]
    B_i = int(batch_info[1])
    W_i = batch_info[2]
    
    # Extract packet size
    L = W_i/B_i
    
    max_compliant = int(bt//L) # Maximum number of compliant pck given the actual state of the bucket
    rem_tokens = bt%L # tokens remaining if max_compliant pcks arrive
    
    if B_i <= max_compliant: # We are not emptying the bucket
        ones = [1] * B_i # ones = [1,1,...] for B_i times
        F.extend(ones) # append B_i ones to F
        bt = rem_tokens + (max_compliant - B_i)*L # Remaining tokens not exploited by source
    
    if B_i > max_compliant: # Some packets will be not compliant
        ones = [1] * max_compliant
        zeros = [0] * (B_i-max_compliant)
        F.extend(ones)
        F.extend(zeros)
        bt = rem_tokens
        
    return bt

def fill_bucket(b_init : int, b, rho, delta_t):
    '''
    Function that computes the final value of b(t) just before the beginning of a new ON period
    '''
    bt = b_init + (rho * delta_t)
    bt = min(b, bt)
    return bt

[1.000000 2.000000 3.000000 4.000000 5.000000 6.000000 7.000000 8.000000
 9.000000 10.000000 11.000000 12.000000 13.000000 14.000000 15.000000
 16.000000 17.000000 18.000000 19.000000 20.000000 21.000000 22.000000
 23.000000 24.000000 26.000000 27.000000 29.000000]
[20072 13482  8835  5834  3923  2618  1742  1157   806   521   319   228
   159    93    61    46    34    26    16     8     6     5     2     2
     2     1     2]


In [8]:
info = S[3,:]
print(info[2])

1000.0


In [9]:
# Final Function for the TBF module
def TBF(b, rho, b0, fname = 'S_matrix.txt'):
    S = read_matrix_S(fname)
    F = []
    bt = b0
    for t in range(S.shape[0]-1):
        bt = count_compliant(F, S,  t, bt) # written F and bt=remaining tokens
        delta_t = S[t+1,0]
        bt = fill_bucket(bt, b, rho, delta_t) # fill with tokens in the OFF period
    F = np.array(F).reshape((-1,1))
    return F

In [10]:
F = TBF(b=3000, rho=8000, b0=3000, fname = 'S_matrix.txt')
print(F.shape)

(179355, 1)


# TODO 3: Empirical Estimates

In [11]:
def estimate_prob(F):
    unique, freq = np.unique(F, return_counts=True)
    freq = np.divide(freq, F.shape[0])
    print(f'{unique} have respectively empirical relative frequence of: {freq}')
    return None

In [12]:
# B = np.arange(2000,50000,2000)
# print(B.shape)
# RHO = np.arange(2000, 50000, 2000)
# z = np.zeros((B.shape[0], RHO.shape[0]))

# for b in range(B.shape[0]):
#     for rho in range(RHO.shape[0]):
#         z[m,l] = (1-np.exp(-0.00082*(b+rho)))*np.exp(-b/rho)

# MM, LL = np.meshgrid(B, RHO)

# fig = plt.figure()
# ax = plt.axes(projection="3d")
# ax.plot_wireframe(MM, LL, z, color='green')
# plt.show()

In [13]:
F = TBF(b=5678, rho=28395, b0=5678)
estimate_prob(F)

[0 1] have respectively empirical relative frequence of: [0.133880 0.866120]


In [14]:
F = TBF(b=5678, rho=283900, b0=5678)
estimate_prob(F)

[0 1] have respectively empirical relative frequence of: [0.131917 0.868083]


In [15]:
# proposed experiment
F = TBF(b=3000, rho=8000, b0=3000)
estimate_prob(F)

[0 1] have respectively empirical relative frequence of: [0.461398 0.538602]
